<a href="https://colab.research.google.com/github/mochangil/K-water_Drought/blob/main/K_water_DroughtInfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2007~2020까지의 가뭄 발생에 대한 피해내역
공공데이터포털의 가뭄피해내역 csv파일 이용 (raw data)

피해종료일이 0인 것과 피해인구가 0인 자료들은 이상치로 여겨 제외했습니다.

#드라이브 마운트

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
Drought_df = pd.read_csv('/content/drive/MyDrive/K-water/한국수자원공사가뭄피해내역.csv',encoding='cp949')

In [ ]:
Drought_df.columns

#데이터 전처리

In [ ]:
#데이터 전처리 - 피해시작일, 피해종료일의 str to datetime, 피해지속기간 column 추가,
# 피해종료일 누락 제외, 피해인구 0 제외
def data_preprocessing():
  Drought_df['피해시작일']=pd.to_datetime(Drought_df['피해시작일'])
  Drought_df['피해종료일']=pd.to_datetime(Drought_df['피해종료일'])
  Drought_df['피해지속기간'] = Drought_df['피해종료일']-Drought_df['피해시작일']
for i in range(len(Drought_df['피해인구'])):
    if Drought_df['피해인구'][i] == 0:
      print(i)
      Drought_df.drop(i)
      i -= 1

data_preprocessing()
Drought_df

In [6]:
Drought_df
Drought_df['years']=Drought_df['피해시작일'].dt.year

#구현할 api들
0. 년도별, 시도별, 시군구별, 읍면동별 검색
1. 년도별 발생지역 리스트
2. 년도별, 지역별 피해지속기간 리스트
3. (년도통합) 지역별 피해총인구, 피해총기간, 발생건수

#0 to 2 : 년도별 구분

In [ ]:
#1. 년도별 발생지역 리스트

# loc 변수를 시도, 시군구, 읍면동 바꿈에 따라 나오는 리스트가 바뀐다.

# 검색하고싶은 년도 입력
year = 2018

def location_per_year(currentyear):
  is_exist =[]
  # loc=Drought_df.loc[(Drought_df['years']==currentyear)]['시도']
  loc=Drought_df.loc[(Drought_df['years']==currentyear)]['시군구']
  #loc=Drought_df.loc[(Drought_df['years']==currentyear)]['읍면동']
  for i in loc:
    if i not in is_exist :
      is_exist.append(i)
    else:
      continue
  return is_exist

# print(location_per_year(year))


In [ ]:
datas = pd.DataFrame(columns=['년도','지역', '총인구(명)', '횟수', '총기간(일)','최대피해일수'])
datas.loc[0]=['년도','지역', '총인구(명)', '횟수', '총기간(일)','최대피해일수']
# print(datas)

   년도  지역  총인구(명)  횟수  총기간(일)  최대피해일수
0  년도  지역  총인구(명)  횟수  총기간(일)  최대피해일수


In [ ]:
#2. 년도별, 지역별 피해지속기간 리스트

# 검색할 년도 입력

count = 1

for year in range(2007,2021):
  list1 = location_per_year(year)
  for i in list1:
    days_list = []
    pop_list = []
    location_df = Drought_df.loc[(Drought_df['시군구']==i)]
    highloc_list = location_df.loc[(location_df['years']==year)]['시도']
    days_list = location_df.loc[(location_df['years']==year)]['피해지속기간'].dt.days
    pop_list = location_df.loc[(location_df['years']==year)]['피해인구']

    length = len(days_list)+1
    total = sum(days_list)
    longest = max(days_list)
    shortest = min(days_list)
    pop = max(pop_list)

    print(length,total,longest,shortest,pop)
    totalpop = max(pop_list)
    j = 0
    # for j in range(len(days_list)):
    #   totalpop += days_list[j] * pop_list[j]
    # datas.loc[count]=[year,i,pop,length,total]
    count += 1

print(datas)
  # print(i)
  # print("\n총",length,"개 지역\n","최장가뭄지속기간",longest,"\n","최단 가뭄지속기간",shortest,"\n")
  # print("\n총",total,"일동안 지속\n")
  # print("평균",int(total/length),"일동안 지속\n")
  # print("next\n\n\n\n\n\n")

#3: 지역별 년도통합 자료



In [7]:
Drought_df['지역명']=Drought_df['시도']+" "+Drought_df['시군구']

In [8]:
is_exist=[]
for i in range(len(Drought_df['지역명'])):
  if Drought_df['지역명'][i] in is_exist:
    continue
  else:
    is_exist.append(Drought_df['지역명'][i])

In [9]:
datas = pd.DataFrame(columns=['지역', '총 피해인구(명)', '총 발생건수', '총기간(일)','최대피해일수(건)','최대피해인구(건)'])
datas.loc[0]=['지역', '총 피해인구(명)', '총 발생건수', '총기간(일)','최대피해일수(건)','최대피해인구(건)']
print(datas)

   지역  총 피해인구(명)  총 발생건수  총기간(일)  최대피해일수(건)  최대피해인구(건)
0  지역  총 피해인구(명)  총 발생건수  총기간(일)  최대피해일수(건)  최대피해인구(건)


In [ ]:
count = 0
for i in is_exist:
  count += 1
  population = Drought_df.loc[(Drought_df['지역명']==i)]['피해인구']
  day = Drought_df.loc[(Drought_df['지역명']==i)]['피해지속기간']
  totpop = sum(population)
  maxpop = max(population)
  cases = len(day)

  totday = sum(day.dt.days) + cases
  maxday = max(day.dt.days) + 1
  # print("인구",population,"\n날짜",days,"\n건수",len(days))
  print(totpop, totday,len(population),len(day),maxpop,maxday)
  datas.loc[count]=[i,totpop,len(day),totday,maxday,maxpop]

In [ ]:
print(datas)

# 데이터 점수화

In [12]:
datas = datas.drop(0)

In [13]:
maxcases = max(datas['총 발생건수'])
maxpopulation = max(datas['총 피해인구(명)'])
maxdays = max(datas['총기간(일)'])

In [14]:
datas['발생건수-백분위'] = datas['총 발생건수']/maxcases
datas['피해인구-백분위'] = datas['총 피해인구(명)']/maxpopulation
datas['총기간-백분위'] = datas['총기간(일)']/maxdays


In [15]:
datas = datas.round({'발생건수-백분위':4,'피해인구-백분위':4,'총기간-백분위':4})

In [16]:
datas['rank-발생건수'] = datas['발생건수-백분위'].rank(ascending=False)
datas['rank-피해인구'] = datas['피해인구-백분위'].rank(ascending=False)
datas['rank-총기간'] = datas['총기간-백분위'].rank(ascending=False)


In [17]:
datas['points'] = datas['발생건수-백분위']*10 + datas['피해인구-백분위']*10 + datas['총기간-백분위']*10

In [18]:
datas['RANK'] = datas['points'].rank(ascending=False)

In [22]:
datas

,지역,총 피해인구(명),총 발생건수,총기간(일),최대피해일수(건),최대피해인구(건),발생건수-백분위,피해인구-백분위,총기간-백분위,rank-발생건수,rank-피해인구,rank-총기간,points,RANK
1,전라남도 완도군,60459,35,5252,364,4978,0.248227,0.366832,0.699521,15.0,2.0,2.0,13.145792,2.0
2,경상북도 의성군,6847,40,419,92,4018,0.283688,0.041544,0.055807,13.5,12.0,27.0,3.810389,19.0
3,전라남도 고흥군,3225,20,2286,366,1000,0.141844,0.019568,0.304475,24.5,18.0,5.0,4.658867,15.0
4,강원도 홍천군,2249,27,397,263,1300,0.191489,0.013646,0.052877,21.0,29.0,29.0,2.58012,25.0
5,경상남도 의령군,523,6,303,80,158,0.042553,0.003173,0.040357,54.0,53.0,39.0,0.860834,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,경상남도 밀양시,24,1,1,1,24,0.007092,0.000146,0.000133,92.5,94.0,96.0,0.07371,96.0
94,전라북도 남원시,1131,7,7,1,357,0.049645,0.006862,0.000932,50.5,37.0,85.5,0.5744,59.0
95,경상남도 함양군,1831,2,2,1,1783,0.014184,0.011109,0.000266,82.5,30.0,92.5,0.255603,86.0
96,경상북도 문경시,20,1,1,1,20,0.007092,0.000121,0.000133,92.5,96.0,96.0,0.073467,97.0


In [24]:
datas1 = datas.sort_values(by=['RANK'],axis=0)

In [ ]:
datas1

# 데이터 자료화

In [26]:
# datas.to_csv('/content/drive/MyDrive/K-water/datas.csv',header=False,index=False,encoding='utf-8-sig')
datas1.to_csv('/content/drive/MyDrive/K-water/rankingdatas.csv',header=False,index=False,encoding='utf-8-sig')